In [3]:
from sklearn.cluster import AgglomerativeClustering
from _jsonnet import evaluate_file as jsonnet_evaluate_file
import json
from transformers import EncoderDecoderModel, logging, BertTokenizer

from models.bottleneck_encoder_decoder import BottleneckEncoderDecoderModel
from utils.clustering_utils import calc_clustering_metrics, get_text_to_vector_func

logging.set_verbosity_info()

**Parameters**

In [5]:
config = json.loads(jsonnet_evaluate_file('../configs/gen_title_v2.jsonnet'))

max_tokens_text = config.pop('max_tokens_text')
max_tokens_title = config.pop('max_tokens_title')
text_to_vec_func = 'bert-FirstCLS'
agency_list = ["ТАСС", "РИАМО", "RT на русском", "Новости Мойка78"]

### Load model & tokenizer

In [6]:
model_path = '/data/aobuhtijarov/models/gen_title_bottleneck_v2/checkpoint-10000/'
gold_markup_file = '/data/aobuhtijarov/datasets/telegram_news/ru_pairs_raw_markup.tsv'
train_file = '/data/aobuhtijarov/datasets/telegram_news/ru_tg_1101_0510.jsonl'
tokenizer_model_path = '/data/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt'

In [7]:
model = BottleneckEncoderDecoderModel.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(tokenizer_model_path, do_lower_case=False, do_basic_tokenize=False)
setattr(tokenizer, 'max_tokens_text', max_tokens_text)

loading configuration file /data/aobuhtijarov/models/gen_title_bottleneck_v2/checkpoint-10000/config.json
Model config EncoderDecoderConfig {
  "architectures": [
    "BottleneckEncoderDecoderModel"
  ],
  "decoder": {
    "_name_or_path": "/data/aobuhtijarov/models/pretrained_dec_6_layers",
    "add_cross_attention": true,
    "architectures": [
      "BertModel"
    ],
    "attention_probs_dropout_prob": 0.2,
    "bad_words_ids": null,
    "bos_token_id": null,
    "chunk_size_feed_forward": 0,
    "decoder_start_token_id": null,
    "directionality": "bidi",
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "eos_token_id": null,
    "finetuning_task": null,
    "gradient_checkpointing": false,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.2,
    "hidden_size": 768,
    "id2label": {
      "0": "LABEL_0",
      "1": "LABEL_1"
    },
    "initializer_range": 0.02,
    "intermediate_size": 3072,
    "is_decoder": true,
    "is_encoder_decod

### Prepare train dataset

In [8]:
from custom_datasets.agency_title_dataset import AgencyTitleDataset
from readers.tg_reader import tg_reader
import tqdm
import random
import numpy as np

In [9]:
train_records = [r for r in tqdm.tqdm(tg_reader(train_file, agency_list))]

18032it [04:23, 68.36it/s] 


In [10]:
text_to_vector_func = get_text_to_vector_func(text_to_vec_func, model, tokenizer)

In [11]:
train_dataset = AgencyTitleDataset(
    train_records,
    tokenizer,
    agency_list,
    max_tokens_text=max_tokens_text,
    max_tokens_title=max_tokens_title
)

In [20]:
total_articles = len(train_dataset)
print('Number of articles in the dataset:', total_articles)

Number of articles in the dataset: 18032


In [13]:
train_embeds = np.empty((total_articles, 768))

for i in tqdm.trange(total_articles):
    record = train_dataset.get_strings(i)
    text = record["title"] + ' ' + record["text"]
    text = text.lower().replace('\xa0', ' ')
    train_embeds[i] = text_to_vector_func(text).detach().numpy().ravel()

100%|██████████| 18032/18032 [27:20<00:00, 10.99it/s]


### Fitting AgglomerativeClustering

In [14]:
%%time

clustering_model = AgglomerativeClustering(
    n_clusters=None,
    distance_threshold=0.18,
    linkage="single",
    affinity="cosine"
)

clustering_model.fit(train_embeds)
train_labels = clustering_model.labels_

CPU times: user 1min 37s, sys: 815 ms, total: 1min 38s
Wall time: 1min 38s


In [15]:
print('Number of clusters:', len(set(train_labels)))

Number of clusters: 14793


In [16]:
from collections import defaultdict

cluster_to_inds = defaultdict(list)

for i, label in enumerate(train_labels):
    cluster_to_inds[label].append(i)

In [17]:
cluster_to_inds

defaultdict(list,
            {10674: [0],
             14290: [1],
             10263: [2],
             9588: [3],
             9966: [4],
             11987: [5],
             347: [6, 3373],
             12301: [7],
             8788: [8],
             8471: [9],
             806: [10, 1559, 2065, 3069],
             13751: [11],
             148: [12, 1426, 2649, 2793, 3230, 5135, 9328, 11260, 11274],
             2604: [13, 20],
             560: [14, 3302],
             774: [15, 3174],
             11295: [16],
             9303: [17],
             1039: [18, 72, 263, 564],
             2016: [19, 2512],
             10872: [21],
             13581: [22],
             14662: [23],
             13602: [24],
             13983: [25],
             352: [26, 33, 250],
             14430: [27],
             8811: [28],
             1277: [29, 30],
             10851: [31],
             7831: [32],
             11764: [34],
             419: [35, 3246],
             440: [36,
       

In [21]:
def get_articles_summary(inds, dataset):        
    return ''.join([
        f'Agency: {dataset.get_strings(i)["agency"]}\nTitle:\n{dataset.get_strings(i)["title"]}\nText:\n{dataset.get_strings(i)["text"]}\n\n'
        for i in inds
    ])

def dump_clusters(cluster_to_inds, train_dataset):
    with open('clusters.txt', 'w', encoding='utf-8') as f:
        for cl_id in cluster_to_inds:
            f.write(f'\tCluster #{cl_id}:\n\n')
            f.write(get_articles_summary(cluster_to_inds[cl_id], train_dataset))
            f.write('\n\n' + ('-'*50+'\n') * 2 + '\n\n')

In [22]:
dump_clusters(cluster_to_inds, train_dataset)

In [26]:
print(get_articles_summary(cluster_to_inds[148], train_dataset))

Agency: RT на русском
Title:
у гуама произошло землетрясение магнитудой 5,6
Text:
землетрясение магнитудой 5,6 зафиксировано у берегов принадлежащего сша острова гуам в тихом океане. об этом сообщает геологическая служба сша (usgs). эпицентр подземных толчков находился в 41 км к северо-западу от населённого пункта йиго. очаг залегал на глубине около 67 км. данных о возможных пострадавших, разрушениях и об угрозе цунами не поступало. ранее сообщалось, что у побережья фиджи произошло землетрясение магнитудой 5,8.

Agency: ТАСС
Title:
у берегов самоа произошло землетрясение магнитудой 5,6
Text:
тасс, 6 мая. землетрясение магнитудой 5,6 произошло в тихом океане, у берегов островного государства самоа. об этом во вторник сообщила геологическая служба сша. по ее данным, эпицентр находился в 118 км к юго-западу от столицы самоа, города апиа (около 40 тыс. жителей). очаг залегал на глубине 33 км. сведений о пострадавших и разрушениях не поступало. угроза цунами не объявлялась.

Agency: RT на р

In [27]:
cluster_sizes = [len(x) for x in cluster_to_inds.values()]

In [28]:
from collections import Counter

Counter(cluster_sizes)

Counter({1: 13119,
         2: 1145,
         4: 95,
         9: 10,
         3: 288,
         22: 1,
         71: 1,
         7: 19,
         23: 1,
         8: 8,
         6: 30,
         5: 38,
         15: 2,
         16: 1,
         11: 9,
         10: 6,
         12: 4,
         18: 1,
         36: 1,
         25: 1,
         44: 1,
         27: 1,
         13: 4,
         14: 2,
         21: 1,
         32: 2,
         19: 2})

### Presumably good clusters

In [30]:
cnt = 0
for cl_id in cluster_to_inds:
    if len(cluster_to_inds[cl_id]) != len(agency_list):
        continue
        
    a = {train_dataset.get_strings(i)["agency"] for i in cluster_to_inds[cl_id]}
    if a == set(agency_list):
        cnt += 1
        print(get_articles_summary(cluster_to_inds[cl_id], train_dataset))
        print('\n\n' + ('-'*50+'\n') * 2 + '\n\n')

Agency: ТАСС
Title:
в шереметьево совершил экстренную посадку ssj-100 из-за отказа двигателя
Text:
москва, 5 мая. /тасс/. пассажирский самолет sukhoi superjet 100 совершил экстренную посадку в аэропорту шереметьево из-за отказа двигателя. об этом тасс сообщили в экстренных службах. "после вылета в саратов, по предварительным данным, у самолета отказал левый двигатель. экипаж принял решение о возвращении. в 07:16 он совершил экстренную посадку в штатном режиме", - сказал собеседник агентства. пострадавших нет. в экстренных службах уточнили, что ssj-100 вылетел в саратов около 06:45. в воздухе он пробыл около получаса, сделав несколько кругов перед посадкой. самолет будет осмотрен технической службой. причины произошедшего устанавливаются. в новость внесена правка (07:27 мск) - передается с уточнением пункта вылета, верно - саратов.

Agency: Новости Мойка78
Title:
в шереметьево приземлился самолет после отказа левого двигателя
Text:
самолет sukhoi superjet 100, который летел из саратова 

In [31]:
print('Presumably good clusters number:', cnt)

Presumably good clusters number: 9
